# 线性回归的简洁实现，使用深度学习框架

In [239]:
import numpy as np
import torch
from torch.utils import data
import matplotlib.pyplot as plt

#创建数据集--------使用线性模型参数：w=[2.-3.4],b=4.2和噪声生成数据集以及标签
def synthetic_data(w,b,num_examples):
    #特征矩阵
    X = torch.normal(0,1,(num_examples, len(w)))
    #标签向量
    y = torch.matmul(X,w) + b
    #加噪声
    y += torch.normal(0,0.01,y.shape)
    return X, y.reshape((-1,1))
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w,true_b,1000)


In [240]:
#调用现有框架中的API来读取数据
def load_array(data_arrays, batch_size,is_train=True):
    '''构造一个pytorch数据迭代器'''
    #使用框架传进入x和y的list，得到dataset，*表示解包
    dataset = data.TensorDataset(*data_arrays)
    #使用dataloader函数，每次从中随机挑选batch
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features,labels), batch_size)


#转成python的iterater，通过next函数得到一个x一个y
print('__iter__' in dir(data_iter))
next(iter(data_iter))


True


[tensor([[-1.6966, -1.2396],
         [ 0.5990,  0.7415],
         [-0.0834,  0.1202],
         [-0.2868,  1.2488],
         [-0.3482, -0.1915],
         [ 0.3913, -0.5989],
         [ 0.2160, -0.8920],
         [-0.3599,  1.1397],
         [ 1.0170, -0.6592],
         [ 0.1337, -0.0825]]),
 tensor([[ 5.0123],
         [ 2.8949],
         [ 3.6035],
         [-0.6198],
         [ 4.1634],
         [ 7.0286],
         [ 7.6643],
         [-0.3938],
         [ 8.4676],
         [ 4.7485]])]

In [241]:
#使用框架的预定义好的层
'''nn是神经网络的缩写
#在PyTorch中，全连接层在Linear类中定义。 
值得注意的是，我们将两个参数传递到nn.Linear中。
第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。
''' 

from torch import nn
#通过linear指定维度。然后放进一个Sequential容器
net = nn.Sequential(nn.Linear(2,1))

#初始化模型参数（第0层）
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


tensor([0.])

In [242]:
#计算均方误差使用的是MSELoss类，也成为平方范数
loss = nn.MSELoss(reduction = 'mean')

#实例化SGD实例:小批量随机梯度下降
trainer = torch.optim.SGD(net.parameters(),lr = 0.03)

In [243]:
#训练过程
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1},loss {l:f}')

epoch 1,loss 0.000240
epoch 2,loss 0.000098
epoch 3,loss 0.000097


In [244]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([0.0004, 0.0004])
b的估计误差： tensor([0.0001])
